In [37]:
import pandas as pd
import numpy as np
import re
df=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/donnees_brutes_2021.csv", sep=";")

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,3,68,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [38]:
p=re.compile('(\D*) \(([0-9]{2,2})[0-9]{3,3}\)')
def ville_dep(x):
    m=p.match(x)
    return m.group(1)+' ' +m.group(2)
df['Ville']=df['Commune'].apply(ville_dep)

In [39]:
var_indicateur=['Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global', 'Indicateur_global_declare']
def yo_float(x):
    return float(str(x).replace(',','.'))

def indicateurs_nombres(df1, liste_indicateur=var_indicateur):
    for indicateur in var_indicateur :
        if indicateur in df1.columns:
            df1[indicateur]=df1[indicateur].apply(yo_float)
    return df1
df=indicateurs_nombres(df)


In [40]:
liste_var=['Deplacement_a_pied',  'Agencement_voirie', 'conflits_pietons', 'Conducteurs_respectent', 'Circulation_vehicules_motorises',  
 'Marche_securite',  'Securite_rues_services', 'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines', 
 'Acces_pietons', 'Traverse_carrefour', 'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole', 
 'Pistes_cyclables', 'Espaces_pietons', 'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus', 
 'Signaletique_specifique','Amenagements_confort_marcheurs', 'Trottoirs_eclaires', 
 'Stationnement_espaces_pietons', 'Signaletique', 'Resume_condition_marche'] 
df['somme_note']=0
df['nombre_notes_donnees']=0
def to_0(x):
    if x==np.nan:
        return 0
    else : 
        return x
def to_1(x):
    if x==np.nan:
        return 0
    else :
        return 1
for i in liste_var :
    df['somme_note']+=df[i].apply(to_0)
    df['nombre_notes_donnees']+=df[i].apply(to_1)
df['note_marchabilite']=df['somme_note']/df['nombre_notes_donnees']


In [41]:
df2=df.copy()
def tt_dataville_(df, var_note, commune='Ville') :
    liste_ville=df[commune].unique()
    grosse_ville=[]
    for ville in liste_ville :
        df_ville1=df[df[commune]==ville]
        if df_ville1[commune].count()>=40 :
            grosse_ville+=[ville]#on va se concentrer sur les grandes villes
    print(len(grosse_ville))
    dataville=pd.DataFrame()
    dataville[commune]=np.array(grosse_ville)
    for col in var_note:
        liste=[]
        for ville in grosse_ville:
            donnees=df[df[commune]==ville]
            moyenne=np.mean(donnees[col])
            liste.append(moyenne)
        dataville['moyenne'+col]=np.array(liste)
    return dataville
dataville_indic=tt_dataville_(df2, var_note=['Indicateur_global', 'Indicateur_securite', 'Indicateur_confort', 'Indicateur_ressenti_global', 'Indicateur_effort_ville','Indicateur_amenagements_services', 'Resume_condition_marche'])
dataville_indic
    

199


,Ville,moyenneIndicateur_global,moyenneIndicateur_securite,moyenneIndicateur_confort,moyenneIndicateur_ressenti_global,moyenneIndicateur_effort_ville,moyenneIndicateur_amenagements_services,moyenneResume_condition_marche
0,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857
1,Aix-en-Provence 13,3.253586,3.543715,3.044108,3.380184,2.637097,3.662826,3.479263
2,Aix-les-Bains 73,3.903217,4.012607,3.894505,3.807692,3.619231,4.182051,4.138462
3,Ajaccio 20,2.453236,2.780708,2.247706,2.698777,2.011468,2.527523,2.403670
4,Albi 81,3.486289,3.736111,3.236152,3.659864,3.010204,3.789116,3.632653
...,...,...,...,...,...,...,...,...
194,Vincennes 94,4.025573,4.402778,4.047619,4.040123,3.662037,3.975309,4.407407
195,Vitré 35,3.320939,3.299964,3.333952,3.471861,2.743506,3.755411,3.467532
196,Vitrolles 13,3.322451,3.452465,3.287726,3.380282,2.982394,3.509390,3.464789
197,Vitry-sur-Seine 94,2.673956,3.226608,2.469925,2.877193,1.973684,2.822368,2.394737


In [62]:
df_geo=df[['Code_INSEE', 'Commune_cp', 'Dep', 'Cp', 'Nom_dep', 'Nom_region', 'Ville']].drop_duplicates(subset='Ville')
"""df_geographie=pd.DataFrame()
for i in dataville_indic['Ville']:
    df_geographie.concat(df_geo[df_geo['Ville']==i])"""
dataville_indic=dataville_indic.merge(df_geo, how='left')

In [66]:
dataville_indic=dataville_indic.drop_duplicates(subset='Ville')
dataville_indic=dataville_indic.reset_index()
dataville_indic

,index,Ville,moyenneIndicateur_global,moyenneIndicateur_securite,moyenneIndicateur_confort,moyenneIndicateur_ressenti_global,moyenneIndicateur_effort_ville,moyenneIndicateur_amenagements_services,moyenneResume_condition_marche,Code_INSEE,...,Cp,Nom_dep,Nom_region,lettre_FUBIndicateur_global,lettre_FUBIndicateur_securite,lettre_FUBIndicateur_confort,lettre_FUBIndicateur_ressenti_global,lettre_FUBIndicateur_effort_ville,lettre_FUBIndicateur_amenagements_services,lettre_FUBResume_condition_marche
0,0,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857,35001,...,35690,Ille-et-Vilaine,Bretagne,A+,A+,A,A+,A,A+,A+
1,2401,Aix-en-Provence 13,3.253586,3.543715,3.044108,3.380184,2.637097,3.662826,3.479263,13001,...,13100,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,D,C,E,D,F,C,D
2,30062,Aix-les-Bains 73,3.903217,4.012607,3.894505,3.807692,3.619231,4.182051,4.138462,73008,...,73100,Savoie,Auvergne-Rhône-Alpes,B,B,C,C,C,B,B
3,34287,Ajaccio 20,2.453236,2.780708,2.247706,2.698777,2.011468,2.527523,2.403670,2A004,...,20090,Corse-du-Sud,Corse,F,E,G,F,G,F,F
4,41586,Albi 81,3.486289,3.736111,3.236152,3.659864,3.010204,3.789116,3.632653,81004,...,81000,Tarn,Occitanie,D,C,D,C,E,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,11916613,Vincennes 94,4.025573,4.402778,4.047619,4.040123,3.662037,3.975309,4.407407,94080,...,94300,Val-de-Marne,Île-de-France,B,A,B,B,C,B,A
195,11919529,Vitré 35,3.320939,3.299964,3.333952,3.471861,2.743506,3.755411,3.467532,35360,...,35500,Ille-et-Vilaine,Bretagne,D,D,D,D,E,C,D
196,11925458,Vitrolles 13,3.322451,3.452465,3.287726,3.380282,2.982394,3.509390,3.464789,13117,...,13127,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,D,D,D,D,E,C,D
197,11931083,Vitry-sur-Seine 94,2.673956,3.226608,2.469925,2.877193,1.973684,2.822368,2.394737,94081,...,94400,Val-de-Marne,Île-de-France,F,D,F,E,G,E,F


In [67]:
def to_lettre(x) :
    x=float(x)
    if x>4.6 :
        return 'A+'
    elif x>4.3 :
        return 'A'
    elif x>3.9 :
        return 'B'
    elif x>3.5 :
        return 'C'
    elif x>3.1 :
        return 'D'
    elif x>2.7 :
        return 'E'
    elif x>2.3 :
        return 'F'
    elif x>=0 :
        return 'G'
    else :
        return np.nan
    
def lettre(df):
    df['lettre_FUB']=df['moyenneIndicateur_global'].apply(to_lettre)
    return df

In [68]:
var_note=['Indicateur_global', 'Indicateur_securite', 'Indicateur_confort', 'Indicateur_ressenti_global', 'Indicateur_effort_ville','Indicateur_amenagements_services', 'Resume_condition_marche']
for indic in var_note :
    dataville_indic['lettre_FUB'+indic]=dataville_indic['moyenne'+indic].apply(to_lettre)

In [69]:
df_nom_comm=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/population par commune.csv", sep=";", skiprows=10)

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [70]:
def prendre1(x) :
    return str(x)[0]
def prendre2(x):
    return(str(x)[1])
df_nom_comm['Ville']=df_nom_comm['LIBGEO']+' '+df_nom_comm['CODGEO'].apply(prendre1)+df_nom_comm['CODGEO'].apply(prendre2)
df_population=df_nom_comm[['Ville','Population totale']]

In [71]:
listecol=['Indicateur_global', 'Indicateur_securite', 'Indicateur_confort', 'Indicateur_ressenti_global', 'Indicateur_effort_ville','Indicateur_amenagements_services', 'Resume_condition_marche']
liste_col=[]
liste_col_moy=[]
for i in listecol :
    liste_col.append('lettre_FUB'+i)
    liste_col.append('moyenne'+i)
    liste_col_moy.append('moyenne'+i)
for geo in [i for i in df_geo.columns] :
    liste_col.append(geo)

In [72]:
dataexp=dataville_indic[liste_col]
taux_pauvrete=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\base-cc-filosofi-2018_CSV_geo2021\taux_pauvrete.csv', sep=';')
taux_pauvrete_lecture=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\base-cc-filosofi-2018_CSV_geo2021\meta_cc_filosofi_2018_COM-geo2021.csv', sep=';')
taux_pauvrete_lecture['CODGEO']=taux_pauvrete_lecture['COD_MOD'].apply(str)#attention, certains sont pris pour des float et d'autres des strings !
taux_pauvrete['Taux_pauvrete']=taux_pauvrete['TP6018']
taux_pauvrete=taux_pauvrete[taux_pauvrete['TP6018'].notna()]
taux_pauvrete['niveau_vie_median_milliers']=taux_pauvrete['MED18']/1000
taux_pauvrete=taux_pauvrete[['CODGEO','Taux_pauvrete','niveau_vie_median_milliers']]
taux_pauvrete['CODGEO']=taux_pauvrete['CODGEO'].apply(str)
df_pauvrete=taux_pauvrete.merge(taux_pauvrete_lecture[['CODGEO','LIB_MOD']])
df_pauvrete['Ville1']=df_pauvrete['LIB_MOD']

p=re.compile('(\D*) [0-9]{2,2}')
def yo_ville1(x):
    m=p.match(x)
    return m.group(1)
dataexp['Ville1']=dataexp['Ville'].apply(yo_ville1)

df_accident=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/accidents.csv", sep=";")
df_accident=df_accident.rename(columns={'Ville':'Ville2'})
def moins_le (x):
    x=str(x)
    if x.split()[0]=='Le' or x.split()[0]=='La' :
        return x[3:]
    else :
        return x
dataexp['Ville2']=dataexp['Ville'].apply(moins_le)
dataexp=dataexp.merge(df_accident, how='left')
dataexp["nombre_accidents_pietons"] = dataexp["nombre d'accidents pietons"].fillna(0)

In [73]:
df_nom_comm=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/population par commune.csv", sep=";", skiprows=10)

def prendre1(x) :
    if len(str(x))==4 :
        return(str(0))    
    else:
        return str(x)[0]
def prendre2(x):
    if len(str(x))==4 :
        return str(x)[0]
    elif str(x)[1]=='A' or str(x)[1]=='B':
        return str(0)
    else :
        return str(x)[1]
    
df_nom_comm['Ville2']=df_nom_comm['LIBGEO']+' '+df_nom_comm['CODGEO'].apply(prendre1)+df_nom_comm['CODGEO'].apply(prendre2)
df_population=df_nom_comm[['Ville2','Population totale']]

In [74]:
df_population['Ville2']=df_population['Ville2'].apply(moins_le)
dataexp=dataexp.merge(df_population, how='left')
dataexp["nombre_accidents_pietons"] = dataexp["nombre d'accidents pietons"].fillna(0)
dataexp['accident pour 1000 habitants 2015-2019']=(dataexp["nombre_accidents_pietons"]*1000)/dataexp['Population totale']
dataexp['plus_accident']=(dataexp['accident pour 1000 habitants 2015-2019']>42784/67000).apply(float)

for i in liste_col_moy :
    dataexp[i]=(dataexp[i]-1)*4

dataexp=dataexp.merge(df_pauvrete[['Ville1', 'niveau_vie_median_milliers']], how='left')
dataexp["revenu médian en milliers d'euros"]=dataexp['niveau_vie_median_milliers']
dataexp=dataexp[[ 'Population totale', 'accident pour 1000 habitants 2015-2019',"revenu médian en milliers d'euros"]+liste_col]
def round2 (x):
    return round(x,2)

<ipython-input-74-4b704b9ab176>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population['Ville2']=df_population['Ville2'].apply(moins_le)


In [76]:
def compte(x) :
    return sum(df['Ville']==x)
dataexp['nombre répondants']=dataexp['Ville'].apply(compte)
def inv (x):
    return 1/np.sqrt(x)
def ecart(x) :
    return np.std(df[df['Ville']==x]['Indicateur_global'])*4
dataexp['écart-type']=dataexp['Ville'].apply(ecart).round(2)
dataexp['intervalle de confiance']=(dataexp['moyenneIndicateur_global']-(dataexp['écart-type']*1.96*dataexp['nombre répondants'].apply(inv))).round(2).apply(str)+' ; '+(dataexp['moyenneIndicateur_global']+(dataexp['écart-type']*1.96*dataexp['nombre répondants'].apply(inv))).round(2).apply(str)
def to_march(x):
    villmarch=df[df['Ville']==x]
    nbremarch=sum(villmarch['Membre_association_marche']=='Oui')
    return nbremarch/(nbremarch+sum(villmarch['Membre_association_marche']=='Non'))
dataexp["pourcentage de membres d'associations de marche"]=dataexp['Ville'].apply(to_march).round(2)
dataexport=dataexp[['Ville', 'Population totale',
       'accident pour 1000 habitants 2015-2019',
       "revenu médian en milliers d'euros", 'nombre répondants', 'écart-type',
       'intervalle de confiance', "pourcentage de membres d'associations de marche"]+liste_col]

In [89]:
dataexport["marge d'erreur"]=dataexp['écart-type']*1.96*dataexp['nombre répondants'].apply(inv)

<ipython-input-89-4f26a79fb5a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataexport["marge d'erreur"]=dataexp['écart-type']*1.96*dataexp['nombre répondants'].apply(inv)


In [90]:
dataexport["marge d'erreur"]

0      0.728000
1      0.465409
2      0.984588
3      0.535781
4      1.042219
         ...   
198    0.818837
199    0.730396
200    0.848705
201    0.593855
202    1.080800
Name: marge d'erreur, Length: 203, dtype: float64

In [87]:
dataexport['intervalle de confiance']

0      14.18 ; 15.63
1        8.55 ; 9.48
2       10.63 ; 12.6
3        5.28 ; 6.35
4        8.9 ; 10.99
           ...      
198    11.28 ; 12.92
199     8.55 ; 10.01
200     8.44 ; 10.14
201       6.1 ; 7.29
202     8.16 ; 10.32
Name: intervalle de confiance, Length: 203, dtype: object

In [91]:
dataexport.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes_villes50.csv', index=False, sep=';')